In [1]:
import numpy as np

# helper functions
def factorial(x):
    factorial = 1
    for i in range(1, x+1):
        factorial *= i
    return float(factorial)

In [2]:
basis = np.identity(3)

R3 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
R3x3 = np.kron(basis, basis).reshape(9, 3, 3)
R_ij = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(np.einsum('ijk, i -> jk', R3x3, R_ij.flatten()), np.einsum('ijk, jk -> i', R3x3, R_ij).reshape(3, 3), sep='\n\n')

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


# LEVI-CIVITA / PERMUTATION TENSOR PROPERTIES

    This is the formula for calculating the indices of the Levi-Civita Tensor

In [3]:
def epsilon_permutation(i, j, k):
    return 0.5 * (j - i) * (k - i) * (k - j)

def levi_civita(e):
    
    lc_tensor = np.zeros((e, e, e))
    
    for i in range(e):
        for j in range(e):
            for k in range(e):
                
                lc_tensor[i][j][k] = epsilon_permutation(i, j, k)
    
    return lc_tensor

e = levi_civita(3)

print(e)

[[[ 0.  0.  0.]
  [-0.  0.  1.]
  [-0. -1.  0.]]

 [[ 0. -0. -1.]
  [ 0.  0.  0.]
  [ 1. -0.  0.]]

 [[ 0.  1. -0.]
  [-1.  0. -0.]
  [ 0.  0.  0.]]]


    Properties of Levi-Civita

In [4]:
print( "Full inner-product of indices size n will equal n-factorial: ", np.einsum('ijk, ijk -> ...', e, e) )

Sym = np.array([[0, 2, 3],
                [2, 0, 4],
                [3, 4, 0]])

print( "Double inner-product with contraction on symetric matrix will equal 0: ", np.einsum('ijk, jk -> ...', e, Sym) )

V = np.array([1, 2, 3])

a = np.einsum('ijk, i -> jk', e, V)
b = np.einsum('ij, j -> ...', a, V)

print( "Product with two of the same vector will be zero: ", b)

print( "Double inner-product with two Levi-Civita tensors is 2 * Kronecker-Delta: ", np.einsum('ijk, ljk -> il', e, e), sep='\n\n')

Full inner-product of indices size n will equal n-factorial:  6.0
Double inner-product with contraction on symetric matrix will equal 0:  0.0
Product with two of the same vector will be zero:  0.0
Double inner-product with two Levi-Civita tensors is 2 * Kronecker-Delta: 

[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 2.]]


    Levi-Civita with Kronecker-Delta:

In [5]:
I = np.identity(3)

print("Full-Contraction with Kronecker-Delta and Levi-Civita tensor equals 0: ",
     np.einsum('ijk, ij -> ...', e, I),
     np.einsum('ijk, ik -> ...', e, I),
     np.einsum('ijk, jk -> ...', e, I),
     sep='\n\n')

Full-Contraction with Kronecker-Delta and Levi-Civita tensor equals 0: 

0.0

0.0

0.0


# Tensor Operations

In [6]:
print( "Determinant of an N x N matrix can be given as e_ijk e_lmn A_il A_jm A_kn * 1/n!: " )

# determinant = 0
A = np.array([[1, 2, 3],
              [4, 5, 6], 
              [7, 8, 9]])

x = np.einsum('ijk, li -> ljk', e, A)
y = np.einsum('ijk, lj -> ilk', x, A)
z = np.einsum('ijk, lk -> ijl', y, A)

det = np.einsum('ijk, ijk -> ...', e, z) * 1/factorial(3)

print('\nDeterminant 1: ', det)

# determinant = 6
A = np.array([[2, 5, 6],
              [4, 5, 6], 
              [7, 8, 9]])

x = np.einsum('ijk, li -> ljk', e, A)
y = np.einsum('ijk, lj -> ilk', x, A)
z = np.einsum('ijk, lk -> ijl', y, A)

det = np.einsum('ijk, ijk -> ...', e, z) * 1.0/factorial(3)

print('\nDeterminant 2: ', det)

Determinant of an N x N matrix can be given as e_ijk e_lmn A_il A_jm A_kn * 1/n!: 

Determinant 1:  0.0

Determinant 2:  6.0


    Vector Products

In [7]:
V_i = np.array([1, 2, 3])
V_j = np.array([3, 9, 2])
V_k = np.array([7, 5, 1])

# Cross Product
X_ik = np.einsum('ijk, j -> ik', e, V_i)
print("Cross-product of same vector: ", np.einsum('ik, k -> i', X_ik, V_i))
print("Cross-product of diff vector: ", np.einsum('ik, k -> i', X_ik, V_k))

# Scalar Triple Product
X_jk = np.einsum('ijk, i -> jk', e, V_i)
X_k  = np.einsum('jk, j -> k', X_jk, V_j)
print("Triple Scalar Product: ", np.einsum('k, k -> ...', X_k, V_k))

V_l, V_m = V_j, V_k # renaming indices

# Vector Triple Product
X_jk = np.einsum('ijk, i -> jk', e, V_i)
X_km = np.einsum('klm, l -> km', e, V_l)
X_k  = np.einsum('km, m -> k', X_km, V_m)
print("Triple Vector Product: ", np.einsum('jk, k -> j', X_jk, X_k))

Cross-product of same vector:  [0. 0. 0.]
Cross-product of diff vector:  [-13.  20.  -9.]
Triple Scalar Product:  -123.0
Triple Vector Product:  [129. -45. -13.]


# Differentiation

In [8]:
def p_dif(x, y):
    
    comparison = x == y
    
    if comparison.all():
        return np.identity(3)
    else:
        return np.zeros((3, 3))
    
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def deriv_logistic(x):
    return x * (1.0 - x)

In [9]:
Y = np.einsum('i, i -> ...', V_i, V_i)
Z = np.einsum('i, i -> ...', V_i, V_j)

# Product Rule
dY_V_i = np.einsum('i, ij -> j', V_i, p_dif(V_i, V_i)) + np.einsum('ij, j -> i', p_dif(V_i, V_i), V_i) # using same vector
dZ_V_i = np.einsum('i, ij -> j', V_i, p_dif(V_i, V_j)) + np.einsum('ij, j -> i', p_dif(V_i, V_i), V_j) # using diff vector

# Quadratic Derivative
Quad = V_i.dot(A).dot(V_i)
dQ_V_i = np.einsum('i, ij -> j', V_i.dot(A), p_dif(V_i, V_i)) + np.einsum('ij, j -> i', p_dif(V_i, V_i), A.dot(V_i)) # using same vector

In [10]:
# Derivative with respect to a matrix

Z = np.array([[2, 1, 6],
              [2, 3, 5],
              [2, 4, 1]])

Z2 = Z.dot(Z.T)

T4 = np.kron(basis, basis).reshape(3, 3, 3, 3) # rank 4 tensor

A1 = np.einsum('ij, kjpq -> ikpq', Z, T4) # if derivative was transposed, that shows up in the indices of the 4D Tensor
B1 = np.einsum('ijpq, jk -> ikpq', T4, Z.T)

d1 = A1 + B1

A1 = np.einsum('ij, jkpq -> ikpq', Z2, T4)
B1 = np.einsum('ijpq, jk -> ikpq', d1, Z)

d1 = A1 + B1

d2 = np.einsum('ijkl -> kl', d1)

print(d2)

[[198. 231. 311.]
 [208. 243. 327.]
 [150. 179. 251.]]
